### Data Lake Gen2 내 csv 파일 processing

In [2]:
%scala
val storageAccountName = "flightdatagen2"
val appID = "873f2a5b-4077-4851-b601-3fb3901632c7"
val password = "[RWhJyYOawpNNCEJmbs8RtrbE.p*:340"
val fileSystemName = "data-hol"
val tenantID = "72f988bf-86f1-41af-91ab-2d7cd011db47"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + password + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

In [3]:
df = sqlContext.read.format('csv').load('abfss://data-hol@flightdatagen2.dfs.core.windows.net/Lab01_flights.csv')

In [4]:
flightDF = df.toDF('Month', 'DayofMonth', 'DayofWeek', 'DepTime', 'ArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted')

flightDF.registerTempTable("flightTable")

####<분석 요건> 1500 마일을 초과하는 항공편의 항공기 모델명과 운항 횟수

In [6]:
%sql
SELECT TailNum, count(TailNum) as cnt 
FROM ( 
    SELECT TailNum 
    FROM flightTable 
    WHERE Distance > 1500 
) longDist 
GROUP BY TailNum
ORDER BY cnt DESC

In [7]:
planeDataDF = sqlContext.read.format('csv').option("header", "true").load('abfss://data-hol@flightdatagen2.dfs.core.windows.net/Lab01_plane-data.csv')

In [8]:
display(planeDataDF)

In [9]:
planeDataDF.registerTempTable("planeDataTable")

In [10]:
%sql
SELECT planeDataTable.Model, sum(cnt) as sumofcnt
FROM ( 
    SELECT longDist.TailNum, count(longDist.TailNum) as cnt
    FROM ( 
        SELECT flightTable.TailNum
        FROM flightTable
        WHERE Distance > 1500
    ) longDist
    GROUP BY longDist.TailNum
    ORDER BY cnt DESC
) longDistanceFlight
INNER JOIN planeDataTable
ON longDistanceFlight.TailNum=planeDataTable.Tailnum
WHERE planeDataTable.Model IS NOT NULL
GROUP BY planeDataTable.Model
ORDER BY sumofcnt DESC